# DEMO:Manejo de Credenciales verificables centralizado y descentralizado

Acontinuacion realizaremos una demostracion que ilustra el manejo de credenciales verificables uno centralizado y otro descentralizado.Utilizamos criptografia de clave publica para firmar y verificar estas credenciales.

Simularemos la emisión, verificación, falsificación y revocación (en el caso centralizado) de credenciales digitales.

### Importacion de librerias a trabajar: Importacion de modulos y librerias





In [ ]:
import json  # para manejar JSON
import uuid  # para generar identificadores únicos
import time  # para obtener el tiempo actual
from typing import Dict, Any, Tuple, Set, Optional # para definir tipos

# para crear widgets interactivos ,para mostrar y limpiar salidas
import ipywidgets as widgets 
from IPython.display import display, clear_output 

# para manejar criptografia
from cryptography.hazmat.primitives import hashes, serialization 
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.asymmetric.ec import EllipticCurvePrivateKey, EllipticCurvePublicKey
from cryptography.exceptions import InvalidSignature

#### Constantes

In [ ]:
# Se designan los nombre a los campos usados en los diccioarios que representan credenciales

CREDENTIAL_ID = "credential_id"
USER_ID = "user_id"
SUBJECT = "subject"
CLAIM = "claim"
VALUE = "value"
TIMESTAMP = "timestamp"
MAYOR_DE_EDAD_CLAIM = "mayor_de_edad"

#### Claves y Estructuras Globales

In [ ]:
# --- Claves de la autoridad (modo centralizado) ---
authority_private_key: EllipticCurvePrivateKey = ec.generate_private_key(ec.SECP256R1())
authority_public_key: EllipticCurvePublicKey = authority_private_key.public_key()

# Lista (set) de credenciales revocadas (identificadas por su UUID)
revoked_credentials: Set[str] = set()

# Para “descentralizado”, guardamos la clave pública del usuario en este diccionario simulado de "ledger"
# ledger_users = { usuario_id_str (UUID) : public_key_object }
ledger_users: Dict[str, EllipticCurvePublicKey] = {}

#### Funciones criptográficas genericas

In [ ]:

def generate_credential_id_and_json(credential_data: Dict[str, Any]) -> Tuple[str, bytes]:
    """
    Genera un ID único para la credencial, lo añade al diccionario y serializa a JSON.
    """
    cred_id = str(uuid.uuid4())
    credential_data[CREDENTIAL_ID] = cred_id
    credential_json = json.dumps(credential_data, sort_keys=True).encode('utf-8') # sort_keys para consistencia
    return cred_id, credential_json

In [ ]:
def sign_credential_data(private_key: EllipticCurvePrivateKey, data_bytes: bytes) -> bytes:
    """
    Firma los bytes de datos proporcionados con la clave privada.
    """
    return private_key.sign(data_bytes, ec.ECDSA(hashes.SHA256()))